In [4]:
!pip install folium

In [5]:
import folium
import pandas as pd
import numpy as np
import pyproj
import folium

# 2022년도 인허가난 카페숍들

In [6]:
data_closed = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/cafe_visualization/restarants_ver.csv',
                   usecols=['사업장명','영업상태명','구', '인허가_년', '좌표정보(x)','좌표정보(y)']
                  )
data_closed

,영업상태명,구,인허가_년,사업장명,좌표정보(x),좌표정보(y)
0,폐업,종로구,2023,밀라커피 (Mila Coffee),201168.350187,453134.920223
1,폐업,강남구,2023,샌드랑,206279.206641,443733.802684
2,폐업,강남구,2023,빈스빈스경기도미술관점(세븐스애비뉴)(한시적),205130.591679,445590.096838
3,폐업,강남구,2023,커피크루(한시적),206279.206641,443733.802684
4,폐업,노원구,2023,위드고,205320.284767,461419.881795
...,...,...,...,...,...,...
54973,영업/정상,마포구,2017,굿애프터눈 합정점,192142.271363,449868.964586
54974,영업/정상,마포구,2017,보노보나마포합정점,192142.271363,449868.964586
54975,영업/정상,마포구,2017,셀렉토커피 상암누리꿈스퀘어점,190250.875092,453017.867203
54976,영업/정상,마포구,2016,로벤,192882.211403,449911.392128


In [7]:
#  2022년도이고, 영업/정상인 카페숍
data_closed = data_closed.loc[(data_closed.인허가_년 == 2022) & (data_closed.영업상태명 == '폐업')]
data_closed

,영업상태명,구,인허가_년,사업장명,좌표정보(x),좌표정보(y)
57,폐업,강서구,2022,쿠잉 마곡나루점,184646.580818,451684.726858
58,폐업,강남구,2022,웨스트커프하우스,203328.852708,446639.095949
59,폐업,강서구,2022,미각상점 화곡점,186169.615372,448981.915156
60,폐업,구로구,2022,수목원 쉼터(베이커리카페),184360.781732,442169.979951
61,폐업,강남구,2022,빈스빈스커피 수원화성점(한시적),205130.591679,445590.096838
...,...,...,...,...,...,...
37159,폐업,강서구,2022,쩨리저트,184805.681473,450977.857003
37160,폐업,강서구,2022,주다카롱(judacaron),184844.841587,450680.365609
37161,폐업,서초구,2022,달다곰이 서초점,200440.010812,443124.671418
37162,폐업,구로구,2022,카페 쌀로미,186772.589404,444308.217952


# 2022년도 폐업한 카페숍들

In [8]:
# 문자열이 들어가 있는 경우가 있다. 숫자형으로 데이터 타입을 변환한 후 이상값들은 제거

data_closed['좌표정보(x)'] = pd.to_numeric(data_closed['좌표정보(x)'], errors="coerce")
data_closed['좌표정보(y)'] = pd.to_numeric(data_closed['좌표정보(y)'], errors="coerce")

# data_location = data.dropna()
data_closed.index=range(len(data_closed))
data_closed.tail()

<ipython-input-8-c14a7b80b515>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_closed['좌표정보(x)'] = pd.to_numeric(data_closed['좌표정보(x)'], errors="coerce")
<ipython-input-8-c14a7b80b515>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_closed['좌표정보(y)'] = pd.to_numeric(data_closed['좌표정보(y)'], errors="coerce")


,영업상태명,구,인허가_년,사업장명,좌표정보(x),좌표정보(y)
701,폐업,강서구,2022,쩨리저트,184805.681473,450977.857003
702,폐업,강서구,2022,주다카롱(judacaron),184844.841587,450680.365609
703,폐업,서초구,2022,달다곰이 서초점,200440.010812,443124.671418
704,폐업,구로구,2022,카페 쌀로미,186772.589404,444308.217952
705,폐업,영등포구,2022,프렌즈 이터널 카페,190933.218774,444305.351690


In [9]:
# EPSG:2097에서 EPSG:4326로 변환하기
def project_array(coord, p1_type, p2_type):
    """
    좌표계 변환 함수
    - coord: x, y 좌표 정보가 담긴 NumPy Array
    - p1_type: 입력 좌표계 정보 ex) epsg:5179
    - p2_type: 출력 좌표계 정보 ex) epsg:4326
    """
    p1 = pyproj.Proj(init=p1_type)
    p2 = pyproj.Proj(init=p2_type)
    fx, fy = pyproj.transform(p1, p2, coord[:, 0], coord[:, 1])
    return np.dstack([fx, fy])[0]


In [10]:
# DataFrame -> NumPy Array 변환
coord = np.array(data_closed.iloc[:,-2:])
coord

array([[184646.58081817, 451684.72685838],
       [203328.8527076 , 446639.09594937],
       [186169.61537157, 448981.91515644],
       ...,
       [200440.01081197, 443124.67141841],
       [186772.58940409, 444308.21795193],
       [190933.21877369, 444305.35168954]])

In [11]:
# 좌표계 정보 설정
p1_type = "epsg:2097"
p2_type = "epsg:4326"

# project_array() 함수 실행
result = project_array(coord, p1_type, p2_type)
result

/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
<ipython-input-9-3275f79941a2>:11: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  fx, fy = pyproj.transform(p1, p2, coord[:, 0], coor

array([[126.82411199,  37.56731822],
       [127.03555491,  37.5219807 ],
       [126.84140237,  37.54299047],
       ...,
       [127.002874  ,  37.49032151],
       [126.84830971,  37.50088971],
       [126.89536222,  37.50091438]])

In [12]:
# 경도 위도 좌표계로 변환된 NumPy Array를 DataFrame에 추가
data_closed['경도(x)'] = result[:, 0]
data_closed['위도(y)'] = result[:, 1]

<ipython-input-12-41a2364eb9c8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_closed['경도(x)'] = result[:, 0]
<ipython-input-12-41a2364eb9c8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_closed['위도(y)'] = result[:, 1]


In [13]:
data_closed.tail()

,영업상태명,구,인허가_년,사업장명,좌표정보(x),좌표정보(y),경도(x),위도(y)
701,폐업,강서구,2022,쩨리저트,184805.681473,450977.857003,126.825928,37.560952
702,폐업,강서구,2022,주다카롱(judacaron),184844.841587,450680.365609,126.826377,37.558272
703,폐업,서초구,2022,달다곰이 서초점,200440.010812,443124.671418,127.002874,37.490322
704,폐업,구로구,2022,카페 쌀로미,186772.589404,444308.217952,126.848310,37.500890
705,폐업,영등포구,2022,프렌즈 이터널 카페,190933.218774,444305.351690,126.895362,37.500914


In [14]:
data_closed = data_closed.dropna()
data_closed

,영업상태명,구,인허가_년,사업장명,좌표정보(x),좌표정보(y),경도(x),위도(y)
0,폐업,강서구,2022,쿠잉 마곡나루점,184646.580818,451684.726858,126.824112,37.567318
1,폐업,강남구,2022,웨스트커프하우스,203328.852708,446639.095949,127.035555,37.521981
2,폐업,강서구,2022,미각상점 화곡점,186169.615372,448981.915156,126.841402,37.542990
3,폐업,구로구,2022,수목원 쉼터(베이커리카페),184360.781732,442169.979951,126.821080,37.481586
4,폐업,강남구,2022,빈스빈스커피 수원화성점(한시적),205130.591679,445590.096838,127.055929,37.512521
...,...,...,...,...,...,...,...,...
701,폐업,강서구,2022,쩨리저트,184805.681473,450977.857003,126.825928,37.560952
702,폐업,강서구,2022,주다카롱(judacaron),184844.841587,450680.365609,126.826377,37.558272
703,폐업,서초구,2022,달다곰이 서초점,200440.010812,443124.671418,127.002874,37.490322
704,폐업,구로구,2022,카페 쌀로미,186772.589404,444308.217952,126.848310,37.500890


# map 시각화


In [15]:
import requests
import json

# 서울 행정구역 json raw파일(githubcontent)
r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)

In [17]:
from folium.plugins import MarkerCluster
m = folium.Map(
    location=[37.559819, 126.963895],
    zoom_start=12,
    tiles='cartodbpositron'
)

folium.GeoJson(
    seoul_geo,
    name='지역구'
).add_to(m)

marker_cluster = MarkerCluster().add_to(m)

for lat, long in zip(data_closed['위도(y)'], data_closed['경도(x)']):
    folium.Marker([lat, long], icon = folium.Icon(color="green")).add_to(marker_cluster)
m